 https://www.kaggle.com/competitions/playground-series-s4e7


Описание данных https://www.kaggle.com/datasets/annantkumarsingh/health-insurance-cross-sell-prediction-data/discussion/516324






# <font color='#11a642' size='6'> **Импорт и установка библиотек**

In [ ]:
!pip install category_encoders -q

In [ ]:
from google.colab import drive
import json
import zipfile

import pandas as pd
import numpy as np

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import (train_test_split, GridSearchCV,
                                     HalvingGridSearchCV, RandomizedSearchCV)

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from category_encoders import TargetEncoder, CatBoostEncoder

from sklearn.ensemble import RandomForestClassifier

from sklearn.tree import DecisionTreeClassifier, plot_tree

from sklearn.metrics import (precision_score, recall_score,
                             roc_auc_score, roc_curve, f1_score)

import matplotlib.pyplot as plt
import seaborn as sns


# <font color='#11a642' size='6'> **Загрузка данных**

In [ ]:
def load_dataset(from_kaggle=False) -> pd.DataFrame:
  '''
  Функция скачивает данные с сайта kaggle, если установлен from_kaggle=True,
  инчае архив считывается по ссылке с моегого гугл диска (такой способ удобен тем, у кого нет доступа к kaggle)
  '''
  if from_kaggle:
    # запросит разрешение к гугл диску, необходимо дать это разрешение
    drive.mount('/content/drive')
    # установим kaggle
    !pip install kaggle -q
    !mkdir ~/.kaggle
    # копируем kaggle.json (предварительно, необходимо сгенерить токен на
    # сайте kaggle и сохранить к себе на гугл диск) в папку ~/.kaggle/
    !cp "/content/drive/MyDrive/Colab Notebooks/config/kaggle.json" ~/.kaggle/
    !kaggle competitions download -c playground-series-s4e7
  else:
    !gdown 1HG4oNC-EfEK7DsXDfKKfb5BtgzevyZZO
  # распаковка архива
  zip_ref = zipfile.ZipFile('playground-series-s4e7.zip', 'r')
  zip_ref.extractall()
  zip_ref.close()
  df_train = pd.read_csv('train.csv')
  df_test = pd.read_csv('test.csv')
  df_sample_submission = pd.read_csv('sample_submission.csv')
  return df_train, df_test, df_sample_submission

In [ ]:
df_train, df_test, df_sample_submission = load_dataset(from_kaggle=False)

In [ ]:
# это данные даны для примера, в таком виде необходимо сделать выборку и загрузить на kaggle
#  (для тех, кто хочет увидеть себя в лидерборде и оценить на сколько ваше решение лучше/хуже остальных)
df_sample_submission.head()

In [ ]:
# это тестовые данные, для которых необходимо предсказать target, то есть применить модель,
#  которую вы разработаете на данных df_train
df_test.head()

In [ ]:
# на этих данных дальше необходимо разработать модель
df_train.head()

In [ ]:
df_train.shape, df_train['Response'].mean()

## <font color='#11a642' size='5'> Определим роли для независимых признаков

In [ ]:
id = 'id'
target = 'Response'

In [ ]:
# на основании описание данных https://www.kaggle.com/datasets/annantkumarsingh/health-insurance-cross-sell-prediction-data/discussion/516324
features_cat_from_description = ['Region_Code', 'Vehicle_Age', 'Policy_Sales_Channel']

In [ ]:
features_num = [feature for feature in df_train.select_dtypes(include='number').columns.to_list()
               if feature not in [id, target, *features_cat_from_description]]
features_num

In [ ]:
features_cat = [feature for feature in df_train.columns.to_list() if feature
                not in [id, target, *features_num]]
features_cat

In [ ]:
features_cat_bin = ['Gender', ]

# <font color='#11a642' size='6'> **Функции**

In [ ]:
def calculate_metrics_and_plot_roc(model, X_train, y_train, X_test, y_test):
    # Предсказание вероятностей на обучающей и тестовой выборке
    y_train_proba = model.predict_proba(X_train)[:, 1]
    y_test_proba = model.predict_proba(X_test)[:, 1]

    # Расчет AUC-ROC
    roc_auc_train = roc_auc_score(y_train, y_train_proba)
    roc_auc_test = roc_auc_score(y_test, y_test_proba)
    print(f"ROC-AUC Train: {roc_auc_train:.2f}")
    print(f"ROC-AUC Test: {roc_auc_test:.2f}")

    # Построение ROC-кривой
    fpr_train, tpr_train, _ = roc_curve(y_train, y_train_proba)
    fpr_test, tpr_test, _ = roc_curve(y_test, y_test_proba)
    plt.figure()
    plt.plot(fpr_train, tpr_train, color='blue', lw=2, label='ROC Curve Train')
    plt.plot(fpr_test, tpr_test, color='darkorange', lw=2, label='ROC Curve Test')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve ({type(model).__name__})')
    plt.legend()
    plt.show()

    # Поиск порога, максимизирующего F1-score
    thresholds = np.arange(0.0, 1.0, 0.01)
    f1_scores = [f1_score(y_test, y_test_proba >= t) for t in thresholds]
    optimal_threshold = thresholds[np.argmax(f1_scores)]
    print(f"Optimal Threshold: {optimal_threshold:.2f}")

    # Пересчет метрик с учетом оптимального порога
    y_train_pred_optimal = (y_train_proba >= optimal_threshold).astype(int)
    y_test_pred_optimal = (y_test_proba >= optimal_threshold).astype(int)

    train_precision = precision_score(y_train, y_train_pred_optimal)
    train_recall = recall_score(y_train, y_train_pred_optimal)
    test_precision = precision_score(y_test, y_test_pred_optimal)
    test_recall = recall_score(y_test, y_test_pred_optimal)

    print(f"Optimal Training Precision: {train_precision:.2f}")
    print(f"Optimal Training Recall: {train_recall:.2f}")
    print(f"Optimal Test Precision: {test_precision:.2f}")
    print(f"Optimal Test Recall: {test_recall:.2f}")

    metrics = {
        'params': [
            'Training_Precision', 'Test_Precision',
            'Training_Recall', 'Test_Recall',
            'ROC_AUC_Train', 'ROC_AUC_Test'
        ],
        'values': [
            train_precision, test_precision,
            train_recall, test_recall,
            roc_auc_train, roc_auc_test
        ]
    }
    return metrics

## <font color='#11a642' size='5'> Анализ пропусков


In [ ]:
df_train.isna().sum()

# <font color='#11a642' size='6'> **Разделите данные на трейн и тест**

In [ ]:
%%time
X_train, X_test, y_train, y_test = train_test_split(df_train[[*features_cat, *features_num]],
                                                    df_train[target],
                                                    test_size=0.2,
                                                    stratify=df_train[target],
                                                    random_state=42)
X_train.shape, X_test.shape

# <font color='#11a642' size='6'> **Подготовка данных к обучению**



1.   Обработка категориальных переменных
2.   Обработка пропусков
3.   Стоит ли делать нормализацию данных?



In [ ]:
bin_pipe = Pipeline(
    [
        (
            'ohe',
            OneHotEncoder(drop='first')
        )
    ]
)

In [ ]:
cat_pipe = Pipeline(
    [
        (
            'cat_encoding',
            TargetEncoder()
        )
    ]
)

In [ ]:
data_preprocessor = ColumnTransformer(
    [
        ('bin_pipe', bin_pipe, features_cat_bin),
        ('cat_pipe', cat_pipe, [feature for feature in features_cat if feature not in features_cat_bin]),
    ],
    remainder='passthrough'
)

In [ ]:
# итоговый пайплайн: подготовка данных и модель
pipe_final = Pipeline(
    [
        ('preprocessor', data_preprocessor),
        ('model', RandomForestClassifier(random_state=12345, n_estimators=5,
                                         warm_start=True, oob_score=True))
    ]
)

In [ ]:
pipe_final

# <font color='#11a642' size='6'> **Построение модели случайный лес**

## <font color='#11a642' size='5'> Параметр out-of-bag (OOB)


In [ ]:
%%time
# обучаем модель на тренировочной выборке
pipe_final.fit(X_train, y_train)


`Bootstrap выборка:` Для создания каждого дерева в Random Forest используется метод bootstrap — случайный выбор с заменой из исходного набора данных. Это означает, что около 63% образцов используются для обучения дерева, а оставшиеся 37% образцов называются out-of-bag (OOB) образцами.

`OOB оценка:` OOB образцы для каждого дерева используются для оценки точности модели. Каждое дерево предсказывает метки для своих OOB образцов, и результат усредняется по всем деревьям. Это дает оценку точности модели без необходимости использования отдельного тестового набора.

Хотя OOB оценка обычно хороша, она может быть `менее точной` для некоторых наборов данных по сравнению с использованием отдельного тестового набора.

In [ ]:
pipe_final.steps[1][1]

In [ ]:
# Вывод OOB точности
print(f"OOB Score: {pipe_final.steps[1][1].oob_score_}")


In [ ]:
%%time
rf_1 = calculate_metrics_and_plot_roc(pipe_final, X_train, y_train, X_test, y_test)

## <font color='#11a642' size='5'> Параметр warm_start


Параметр `warm_start` позволяет добавлять новые деревья к уже существующему random forest

In [ ]:
%%time
# Добавление новых деревьев к модели
additional_estimators = 2
pipe_final.steps[1][1].n_estimators += additional_estimators
pipe_final.fit(X_train, y_train)

In [ ]:
# Получаем параметры пайплайна
params = pipe_final.named_steps['model'].get_params()
params

In [ ]:
%%time
rf_2 = calculate_metrics_and_plot_roc(pipe_final, X_train, y_train, X_test, y_test)

In [ ]:
pipe_final.named_steps['preprocessor'].get_feature_names_out()

## <font color='#11a642' size='5'> Построение случайного леса с поиском наилучших гиперапарметров


In [ ]:
param_grid = {
    # 'preprocessor__cat_pipe__cat_encoding':[TargetEncoder(), CatBoostEncoder()],
    'model__max_depth' : [3, 5],
    'model__n_estimators' : [10, 12]
    # 'model__ccp_alpha' : [0.01, 0.1]
}

In [ ]:
grid = HalvingGridSearchCV(
    pipe_final,
    param_grid,
    cv=5,
    scoring='roc_auc',
    n_jobs = -1
)
grid.fit(X_train, y_train)

In [ ]:
# %%time
# grid = RandomizedSearchCV(
#     pipe_final,
#     param_grid,
#     cv=5,
#     scoring='roc_auc',
#     n_jobs = -1
# )
# grid.fit(X_train, y_train)

In [ ]:
df_cv_results = pd.DataFrame(grid.cv_results_)

In [ ]:

print('Лучшая модель и её параметры:\n\n', grid.best_estimator_)
print ('Метрика лучшей модели на тренировочной выборке:', grid.best_score_)


In [ ]:
%%time
rf_3 = calculate_metrics_and_plot_roc(grid.best_estimator_, X_train, y_train, X_test, y_test)

In [ ]:
grid.best_estimator_.named_steps['model'].feature_importances_

In [ ]:
pd.DataFrame(
    {"feature": X_train.columns, "importance": grid.best_estimator_.named_steps['model'].feature_importances_}
).sort_values(by="importance", ascending=False).reset_index(drop=True)

In [ ]:
STOP

# <font color='#11a642' size='6'> **Опционально, для тех, кто хочет сравнить свое решение с другими решениями на kaggle. Применение алгоритма к тестовой выборке df_test и сабмит решения на kaggle**

In [ ]:
prediction = grid.best_estimator_.predict_proba(df_test)

In [ ]:
df_test['Response'] = prediction[:,1]

In [ ]:
df_test[[id, 'Response']].to_csv('/content/submissions.csv', index=False)

In [ ]:
# загрузка данных на kaggle
drive.mount('/content/drive')
# установим kaggle
!pip install kaggle -q
!mkdir ~/.kaggle
# копируем kaggle.json (предварительно, необходимо сгенерить токен на
# сайте kaggle и сохранить к себе на гугл диск) в папку ~/.kaggle/
!cp "/content/drive/MyDrive/Colab Notebooks/config/kaggle.json" ~/.kaggle/
!kaggle competitions submit -c playground-series-s4e7 -f /content/submissions.csv -m "1 submit"